In [1]:
from GSNN import utils
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
import torch

from GSNN.GSNNmodel import GSNN, GCN
from GSNN.GSNNTrain import GSNNTrainer, GCNTrainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
dataset = 'cora' # dataset = {'cora', 'citeseer', 'pubmed'}
seed = 15 
weight_decay = 5e-4
lr = 1e-2
hidden_dim = 16
epochs = 200000
earlystopping = True

In [3]:
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, labels = utils.load_data(dataset)
_A_obs = adj
# _X_obs = utils.preprocess_features(features)
_X_obs = features
_A_obs = _A_obs + _A_obs.T
_A_obs[_A_obs > 1] = 1

# _A_obs.setdiag(0)
_A_obs = _A_obs.astype("float32")
_A_obs.eliminate_zeros()
_X_obs = _X_obs.astype("float32")

_An = utils.preprocess_graph(_A_obs)
split_train, split_val, split_test = np.where(train_mask)[0], np.where(val_mask)[0], np.where(test_mask)[0]
split_unlabeled = np.union1d(split_val, split_test)
_Z_obs = labels
utils.set_seed(42, torch.cuda.is_available())
adj = utils.sparse_mx_to_torch_sparse_tensor(_An).float()
adj_cuda = adj.cuda()

feature_cuda, label_cuda, idx_train_cuda, idx_val_cuda, idx_test_cuda = utils.convert_to_Tensor([_X_obs, _Z_obs, split_train, split_val, split_test])

In [4]:
# An instantiation of GSNN-M model
x_dim = feature_cuda.shape[1]
y_dim = label_cuda.shape[1]
h_dim, z_dim, r_dim = utils.get_h_dim(hidden_dim)

gcn = GCN(x_dim, h_dim, y_dim, True)
gsnn = GSNN(x_dim, y_dim, h_dim, r_dim, z_dim, gcn)

optimizer = torch.optim.Adam(gsnn.parameters(), lr=lr, weight_decay=weight_decay)
gsnntrainer = GSNNTrainer(gsnn, optimizer)

In [5]:
res = gsnntrainer.train(adj_cuda, feature_cuda, label_cuda, idx_train_cuda, idx_val_cuda, idx_test_cuda, epochs, earlystopping=earlystopping)

e:\anaconda\lib\site-packages\torch\nn\functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
G:\opensource\GSNN\GSNN\GSNNTrain.py:102: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  kl_pq += (F.gumbel_softmax(y_encode[non_label].detach(), tau=1.0, hard=True) * (-F.log_softmax(y_pred_total[i][non_label]))).sum()/len(non_label)


Epoch: 0, Train_loss: 13.860234260559082.
--------------------------------------------------
Epoch: 10, Train_loss: 6.2846198081970215.
--------------------------------------------------
Epoch: 20, Train_loss: 5.567917823791504.
--------------------------------------------------
Epoch: 30, Train_loss: 5.0991950035095215.
--------------------------------------------------
Epoch: 40, Train_loss: 4.623395919799805.
--------------------------------------------------
Epoch: 50, Train_loss: 4.309807300567627.
--------------------------------------------------
Epoch: 60, Train_loss: 3.899207830429077.
--------------------------------------------------
Epoch: 70, Train_loss: 3.56889009475708.
--------------------------------------------------
Epoch: 80, Train_loss: 3.4155068397521973.
--------------------------------------------------
Epoch: 90, Train_loss: 3.076070547103882.
--------------------------------------------------
Epoch: 100, Train_loss: 2.864795207977295.
-------------------------